In [ ]:
# para montar el drive al colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Es mas facil subir un zip con todos los archivos del dataset que subir una carpeta, podes usar este comando para extraer archivos zip
#!unzip "/content/drive/MyDrive/Trainer.zip" -d "/content/drive/My Drive/trainer"

In [11]:
# Este comando podes utilizar si los imports te fallan, tenes que pararte en la carpeta donde estan los archivos del trainer
#%cd /content/drive/MyDrive/trainer

/content/drive/MyDrive/trainer/Trainer


In [ ]:
#por si no se instaló tensorflow por defecto
!pip install tensorflow

In [ ]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save'

sess = tf.InteractiveSession()

##### SE PUEDE CAMBIAR #####
L2NormConst = 0.001
############################

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst

##### SE PUEDE CAMBIAR #####
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
############################

sess.run(tf.global_variables_initializer())

tf.summary.scalar("loss", loss)
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

##### SE PUEDE CAMBIAR #####
epochs = 3
batch_size = 32
############################

for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Paso: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Modelo guardado en archivo: %s" % filename)

